In [1]:
import orbipy as op
import numpy as np
from scipy.optimize import bisect
import pickle

from instruments import instruments

In [2]:
model = op.crtbp3_model('Earth-Moon (default)', integrator=op.dopri5_integrator(), stm=True)
one_thousand_kms = (1-model.L1) / 61.350

In [24]:
def calculate_next_point_with_same_jacoby_constant(correction, previous_state, cj, previous_alpha_degrees):
    """
    Given a previous point on the contour line of the jacobi constant, find the next point on the same contour line (with the same energy/jacobi constant).
    :param previous_state: previous (or initial) state of hte contour line point
    :param cj: jacobi constant of the contour line
    :param previous_alpha_degrees: angle alpha in degrees, for approximating the direction
    :return: (state, alpha)
    """
    def find_alpha(alpha_degrees):
        """
        Given angle alpha, calculate the corresponding state (polar coordinates) and returns the difference between given jacobi constant and the calculated one
        """
        state = model.get_zero_state().copy()
        x = previous_state[0] + np.cos(np.radians(alpha_degrees))*dr
        z = previous_state[2] + np.sin(np.radians(alpha_degrees))*dr
        state[[0,2]] = x, z
        state[4] = previous_state[4]
        velocity = correction.calc_dv(0, state)
        state += velocity
        cj_new = model.jacobi(state)
        return cj - cj_new

    # Find angle alpha that corresponds to the lowest possible deviation of jacobi constant between current point and next point (as in, finds the alpha that corresponds to the point lying on the contour line)
    res = bisect(find_alpha, previous_alpha_degrees - 40, previous_alpha_degrees + 40, 
        xtol=1e-6, maxiter=100,
        full_output=True)
    
    target_alpha_degrees = res[0]
    print("cj = "+ str(cj) + "; alpha = " + str(target_alpha_degrees))
    r = model.get_zero_state().copy()
    x = previous_state[0] + np.cos(np.radians(target_alpha_degrees))*dr
    z = previous_state[2] + np.sin(np.radians(target_alpha_degrees))*dr
    r[[0, 2]] = x, z
    r += correction.calc_dv(0, r)
    return r, target_alpha_degrees

def calculate_contour_line_of_jakobi_constant(model, initial_state, alpha):
    """
    Given initial state of the zvl, calculate the contour line.
    :param initial_state: usually a point, lying on the Zero Velocity Line
    :param alpha: initial alpha (0 for right side and negative velocity, 180 for left side and positive velocity)
    :return: array of states [ [x,z,vy], [x,z,vy], [x,z,vy], ... ]
    """
    Cj = model.jacobi(initial_state)
    points = []

    points.append(initial_state[[0, 2, 4]].tolist())
    
    left = op.eventX(model.L1 - 33 * one_thousand_kms)
    right = op.eventX(model.L1 + 55 * one_thousand_kms)

    while points[-1][1] > 0: # until we cross the x axis
        correction = op.border_correction(model, op.y_direction(), left, right)
        try:
            next_point_with_same_jakoby_constant, alpha = calculate_next_point_with_same_jacoby_constant(correction, initial_state, Cj, alpha)
        except (RuntimeError, ValueError) as e:
            print(f"next point from {points[-1]} failed! {e}")
            break
        points.append(next_point_with_same_jakoby_constant[[0, 2, 4]].tolist())
        print(f"    point {points[-1]}")
    return points

In [25]:
data = np.load('../data/contour_points/zvl/zvl_4__0_3_dv.npy')

dr = 0.3*one_thousand_kms
contour_lines_data = []

initial_state_points = np.array(np.meshgrid(np.arange(0, 49), [0, 180])).T.reshape(-1, 2).tolist()
print(initial_state_points)

for point in initial_state_points:
    print(f"Calculating contour line for initial state {point[0]}, {"left" if point[1] == 180 else "right"} side")
    alpha = point[1]
    initial_state = model.get_zero_state()
    initial_state[[0,2]] = data[point[0]][0], data[point[0]][1]
    
    contour_line = calculate_contour_line_of_jakobi_constant(model, initial_state, alpha)
    contour_lines_data.append(contour_line)
    print(contour_line)
    with open("../data/contour_points/contour_points_data_7_points.pickle", 'wb') as handle:
        pickle.dump(contour_lines_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


[[0, 0], [0, 180], [1, 0], [1, 180], [2, 0], [2, 180], [3, 0], [3, 180], [4, 0], [4, 180], [5, 0], [5, 180], [6, 0], [6, 180], [7, 0], [7, 180], [8, 0], [8, 180], [9, 0], [9, 180], [10, 0], [10, 180], [11, 0], [11, 180], [12, 0], [12, 180], [13, 0], [13, 180], [14, 0], [14, 180], [15, 0], [15, 180], [16, 0], [16, 180], [17, 0], [17, 180], [18, 0], [18, 180], [19, 0], [19, 180], [20, 0], [20, 180], [21, 0], [21, 180], [22, 0], [22, 180], [23, 0], [23, 180], [24, 0], [24, 180], [25, 0], [25, 180], [26, 0], [26, 180], [27, 0], [27, 180], [28, 0], [28, 180], [29, 0], [29, 180], [30, 0], [30, 180], [31, 0], [31, 180], [32, 0], [32, 180], [33, 0], [33, 180], [34, 0], [34, 180], [35, 0], [35, 180], [36, 0], [36, 180], [37, 0], [37, 180], [38, 0], [38, 180], [39, 0], [39, 180], [40, 0], [40, 180], [41, 0], [41, 180], [42, 0], [42, 180], [43, 0], [43, 180], [44, 0], [44, 180], [45, 0], [45, 180], [46, 0], [46, 180], [47, 0], [47, 180], [48, 0], [48, 180]]
eventX:[val]np.float64(0.74913597542714

capi_return is NULL
Call-back cb_solout_in___user__routines failed.


KeyboardInterrupt: 

In [ ]:
instruments.ism_contour_visualizer(contour_lines_data).show()